Aquí se tienen los códigos rápidos de las tareas que se pueden utilizar. Se dividen por temas y según el uso en las tareas

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# **CÓDIGOS VARIOS**

In [10]:
def BACKTRAKING(alpha_i: float, p: float, c: float, 
                xk: np.array, f, fxk: np.array,
                gradfxk: np.array, pk: np.array, Nb: int):
    alpha = alpha_i
    for i in range(Nb):
        if f(xk + alpha*pk) <= fxk + c*alpha*(gradfxk.T)@pk:
            return alpha, i
        alpha = p*alpha
    return alpha, Nb

def contornosFnc2D(fncf, xleft, xright, ybottom, ytop, levels,
                    secuencia1 = None, secuencia2 = None):
    ax = np.linspace(xleft, xright, 250)
    ay = np.linspace(ybottom, ytop, 200)
    mX, mY = np.meshgrid(ax, ay)
    mZ = mX.copy()
    for i,y in enumerate(ay):
        for j,x in enumerate(ax):
            mZ[i,j] = fncf(np.array([x,y]))
    
    fig, ax = plt.subplots(figsize=(8, 5))
    CS = ax.contour(mX, mY, mZ, levels, cmap='brg_r')
    if secuencia1 is not None:
        puntos1 = np.array(secuencia1)
        ax.plot(puntos1[:,0], puntos1[:,1], marker = '.',
                    color='#000000', label = r"$\Delta_{max} = 4$")
    if secuencia2 is not None:
        puntos2 = np.array(secuencia2)
        ax.plot(puntos2[:,0], puntos2[:,1], marker = '.', markersize = 3,
                    color='#FF0000', label = r"$\Delta_{max} = 0.25$")
    ax.plot()
    ax.grid(True)
    cbar = fig.colorbar(CS)
    return ax

def BACKTRAKING_WOLF(a_init: float, p: float, c1: float,
                c2: float, xk: np.array, f, gradf,
                dk: np.array, Nb: int):
    a = a_init
    for i in range(Nb): # STRONG / NORMAL
        #if (f(xk + a*dk) <= f(xk) + c1*a*gradf(xk).T @ dk) and (np.abs(gradf(xk + a*dk).T @ dk) <= -c2*gradf(xk).T @ dk):
        if (f(xk + a*dk) <= f(xk) + c1*a*gradf(xk).T @ dk) and (gradf(xk + a*dk).T @ dk >= c2*gradf(xk).T @ dk):
            return a, i
        a = p*a
    return a, Nb

def DESC_MAX_BACKTRACKING(f, gradf, xk: np.array, tol: float,
                        max_iter: int, alpha_i: float,
                        p: float, c: float, Nb: int):
    """
    Gradient Descent with Backtracking Line Search.
    
    Args:
    - f:        function to minimize.
    - gradf:    gradient of f.
    - xk:       initial point.
    - tol:      tolerance.
    - max_iter: maximum number of iterations.
    - alpha_i:  initial value of alpha.
    - p:        factor to reduce alpha.
    - c:        factor to reduce alpha.
    - Nb:       maximum number of iterations for backtracking.

    Returns:
    - xk:  optimal point.
    - k:   number of iterations.
    - T/F: if the algorithm converged, False otherwise.
    """
    for k in range(max_iter):
        gk = gradf(xk)
        pk = -gk
        ak = BACKTRAKING(alpha_i, p, c, xk, f, f(xk), gk, pk, Nb)[0]
        if ak*np.linalg.norm(gk) < tol:
            return xk, k, True
        xk = xk + ak * pk
    return xk, max_iter, False

# **FUNCIONES PRUEBA**

In [8]:
def f_Himmelblau(x: np.array):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2
def grad_Himmelblau(x: np.array):
    x1 = 4*x[0]*(x[0]**2 + x[1] - 11) + 2*(x[0] + x[1]**2 - 7)
    x2 = 2*(x[0]**2 + x[1] - 11) + 4*x[1]*(x[0] + x[1]**2 - 7)
    return np.array([x1,x2], dtype = float)
def Hess_Himmelblau(x: np.array):
    x11 = 12*x[0]**2 + 4*x[1] - 42
    x12 = 4*x[0] + 4*x[1]
    x22 = 4*x[0] + 12*x[1]**2 - 26
    return np.array([[x11, x12], [x12, x22]], dtype = float)

def f_Beale(x: np.array):
    return (1.5 - x[0] + x[0]*x[1])**2 + (2.25 - x[0] + x[0]*x[1]**2)**2 + (2.625 - x[0] + x[0]*x[1]**3)**2
def grad_Beale(x: np.array):
    x1 = 2*(x[1] - 1)*(1.5 - x[0] + x[0]*x[1]) + 2*(x[1]**2 - 1)*(2.25 - x[0] + x[0]*x[1]**2) + 2*(x[1]**3 - 1)*(2.625 - x[0] + x[0]*x[1]**3)
    x2 = 2*x[0]*(1.5 - x[0] + x[0]*x[1]) + 4*x[0]*x[1]*(2.25 - x[0] + x[0]*x[1]**2) + 6*x[0]*(x[1]**2)*(2.625 - x[0] + x[0]*x[1]**3)
    return np.array([x1,x2], dtype = float)
def Hess_Beale(x: np.array):
    x11 = 2*(x[1]**3 - 1)**2 + 2*(x[1]**2 - 1)**2 + 2*(x[1] - 1)**2
    x12 = 4*x[0]*x[1]*(x[1]**2 - 1) + 4*x[1]*(x[0]*x[1]**2 - x[0]+2.25) + 6*x[0]*x[1]**2*(x[1]**3 - 1) + 6*x[1]**2*(x[0]*x[1]**3 - x[0]+2.625) + 2*x[0]*(x[1]-1) + 2*(x[0]*x[1] - x[0]+1.5)
    x22 = 18*x[0]**2*x[1]**4 + 8*x[0]**2*x[1]**2 + 2*x[0]**2 + 12*x[0]*x[1]*(x[0]*x[1]**3 - x[0] + 2.625) + 4*x[0]*(x[0]*x[1]**2 - x[0]+2.25)
    return np.array([[x11, x12], [x12, x22]], dtype = float)

def f_Rosenbrock(x: np.array):
    n = len(x)
    s = 0
    for i in range(n-1):
        s = s + 100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return s
def grad_Rosenbrock(x: np.array):
    n = len(x)
    grad = np.zeros(n)
    grad[0] = -400*x[0]*(x[1] - x[0]**2) - 2*(1-x[0])
    grad[n-1] = 200*(x[n-1] - x[n-2]**2)
    for j in range(1,n-1):
        grad[j] = 200*(x[j]-x[j-1]**2) - 400*x[j]*(x[j+1] - x[j]**2) - 2*(1-x[j])
    return np.array(grad, dtype = float)
def Hess_Rosenbrock(x: np.array):
    n = len(x)
    Hess = np.zeros((n,n))
    Hess[0,0] = -400*(x[1]-x[0]**2) + 800*x[0]**2 + 2
    Hess[1,0] = -400*x[0]
    Hess[n-2,n-1] = -400*x[n-2]
    Hess[n-1,n-1] = 200
    for j in range(1,n-1):
        Hess[j-1,j] = -400*x[j-1]
        Hess[j,j] = -400*(x[j+1]-x[j]**2) +800*x[j]**2 + 202
        Hess[j+1,j] = -400*x[j]
    return np.array(Hess, dtype = float)

n = 10
A1 = n*np.eye(n, dtype = float) + np.ones([n,n], dtype = float)
b1 = np.ones(n, dtype = float)
f_cuad = lambda x: 0.5 * x.T @ A1 @ x - b1.T @ x
gradf_cuad = lambda x: A1 @ x - b1
Hessf_cuad = lambda x: A1

n = 10
A2 = np.zeros((n,n), dtype = float)
for i in range(n):
    for j in range(n):
        A2[i,j] = np.exp(-0.25*(i-j)**2)
b2 = np.ones(n, dtype = float)
f_cuad = lambda x: 0.5 * x.T @ A2 @ x - b2.T @ x
gradf_cuad = lambda x: A2 @ x - b2
Hessf_cuad = lambda x: A2

In [9]:
alpha = np.array([1.0, 1.2, 3.0, 3.2], dtype = float)
A = np.array(
    [[10, 3, 17, 3.5, 1.7, 8],
    [0.05, 10, 17, 0.1, 8, 14],
    [3, 3.5, 1.7, 10, 17, 8],
    [17, 8, 0.05, 10, 0.1, 14]], dtype = float)
P = 10**(-4) * np.array(
    [[1312, 1696, 5569, 124, 8283, 5886],
    [2329, 4135, 8307, 3736, 1004, 9991],
    [2348, 1451, 3522, 2883, 3047, 6650],
    [4047, 8828, 8732, 5743, 1091, 381]], dtype = float)
def f_Hartman(x: np.array):
    result = 0
    for i in range(4):
        inner_sum = 0
        for j in range(6):
            inner_sum += A[i][j] * (x[j] - P[i][j])**2
        result += alpha[i] * np.exp(-inner_sum)
    return -(2.58 + result)/1.94
def grad_Hartman(x: np.array):
    grad = np.zeros(6)
    for k in range(6):
        for i in range(4):
            inner_sum = 0
            for j in range(6):
                inner_sum += A[i][j] * (x[j] - P[i][j])**2
            grad[k] += alpha[i] * A[i][k] * (x[k] - P[i][k]) * np.exp(-inner_sum)
    return np.array(2*grad/1.94, dtype = float)
def Hess_Hartman(x: np.array):
    hess = np.zeros((6, 6))
    for l in range(6):
        for k in range(6):
            for i in range(4):
                inner_sum = 0
                for j in range(6):
                    inner_sum += A[i][j] * (x[j] - P[i][j])**2
                if k != l:
                    hess[l][k] += -2*alpha[i] * A[i][k] * A[i][l] * (x[k]-P[i][k])*(x[l]-P[i][l]) * np.exp(-inner_sum)
                else:
                    hess[l][l] += alpha[i] * A[i][l] * (1-2*A[i][l]*(x[l]-P[i][l])**2) * np.exp(-inner_sum)
    return np.array(2*hess/1.94, dtype = float)

# **UNIDAD 4**

## **4.1**

In [ ]:
def REG_CONF_CAUCHY(f, gradf, Hessf, xk: np.array,
                    maxiter: int, tol: float, Dmax: float,
                    Dmin: float, eta: float):
    """
    Confident region method with Cauchy step.
    Args:
    - f: function to minimize.
    - gradf: gradient of f.
    - Hessf: Hessian of f.
    - xk: initial point.
    - maxiter: maximum number of iterations.
    - tol: tolerance.
    - Dmax: maximum trust region radius.
    - Dmin: minimum trust region radius.
    - eta: parameter for the acceptance of the step.
    Returns:
    - k: number of iterations.
    - xk: final point.
    - gk: gradient at xk.
    - True if the method converged, False otherwise.
    """
    M = None
    if len(xk)==2:
        M = [xk]
    em = np.finfo(float).eps
    Dk = (Dmax + Dmin)/4
    for k in range(maxiter):
        gk = gradf(xk)
        if np.linalg.norm(gk) < tol:
            return k, xk, gk, True, M
        Bk = Hessf(xk)
        if gk.T @ Bk @ gk <= em:
            tk = 1.0
        else:
            tk = min(1.0, np.linalg.norm(gk)**3/(Dk*gk.T @ Bk @ gk))
        pkC = -tk*(Dk/np.linalg.norm(gk))*gk
        pk = (f(xk) - f(xk+pkC)) / (- (pkC.T @ gk + 0.5 * pkC.T @ Bk @ pkC)) # Cambio analitico
        if pk < 0.25 and Dk > 4*Dmin:
            Dk = Dk/4
        elif pk > 0.75 and np.abs(np.linalg.norm(pk) - Dk) < em:
            Dk = min(Dmax, 2*Dk)
        else:
            Dk = Dk
        if pk > eta:
            xk = xk + pkC
        else:
            xk = xk
        if len(xk)==2:
            M.append(xk)
    return maxiter, xk, gk, False, M

## **4.2**

# **UNIDAD 5**

## **5.1**

In [ ]:
def LINEAR_CONJUGATE_GRADIENT(xk: np.array, A: np.array,
                        b: np.array, maxiter: int, tol: float):
    """
    SOLVE THE SYSTEM OF EQUATIONS Ax = b, WHERE A IS A POSITIVE DEFINITE SYMMETRIC MATRIX USING THE CONJUGATE GRADIENT METHOD

    Args:
    - xk:      initial guess.
    - A:       positive definite symmetric matrix.
    - b:       vector b.
    - maxiter: maximum number of iterations.
    - tol:     tolerance.

    Outputs:
    - xk:  approach to the solution.
    - rk:  residual.
    - k:   number of iterations.
    - T/F: if the method converged.
    """
    rk = A @ xk - b
    pk = - rk
    for k in range(maxiter+1):
        if np.linalg.norm(rk) < tol:
            return xk, rk, k, True
        ak = (rk.T @ rk) / (pk.T @ A @ pk)
        xk = xk + ak * pk
        rk_new = rk + ak * A @ pk
        bk = (rk_new.T @ rk_new) / (rk.T @ rk)
        pk = - rk_new + bk * pk
        rk = rk_new
    return xk, rk, maxiter, False

## **5.2**

In [ ]:
def NONLINEAR_CONJUGATE_GRADIENT_FR(xk: np.array, f, gradf, maxiter: int,
                                tol: float, a_init: float, p: float,
                                c1: float, c2: float, Nb: int):
    """
    THIS FINDS THE MINIMIZER OF f USING THE NON-LINEAR CONJUGATE GRADIENT METHOD WITH THE FLETCHER-REEVES FORMULA.

    Args:
    - xk:      initial guess.
    - f:       function to minimize.
    - gradf:   gradient of the function to minimize.
    - maxiter: maximum number of iterations.
    - tol:     method tolerance.
    - a_init:  initial value for the step size in backtracking.
    - p:       reduction factor for the step size in backtracking.
    - c1:      parameter for the sufficient descent condition.
    - c2:      parameter for the curvature condition.
    - Nb:      maximum number of iterations in backtracking.
    
    Outputs:
    - xk:  approach to the minimizer of f.
    - gk:  gradient of f at xk.
    - k:   number of iterations.
    - T/F: if the method converged.
    - nr:  number of restarts (bk = 0).
    """
    gk = gradf(xk)
    dk = -gk
    nr = 0
    for k in range(maxiter + 1):
        if np.linalg.norm(gk) < tol:
            return xk, gk, k, True, nr
        ak, k1 = BACKTRAKING_WOLF(a_init = a_init, p = p, c1 = c1, c2 = c2,
                                xk = xk, f = f, gradf = gradf, dk = dk, Nb = Nb)
        xk = xk + ak*dk
        gk_n = gradf(xk)
        if np.abs(gk_n.T @ gk) < 0.2*np.linalg.norm(gk_n)**2:
            bk = (gk_n.T @ gk_n) / (gk.T @ gk)
        else:
            bk = 0
            nr += 1
        dk = -gk_n + bk*dk
        gk = gk_n
    return xk, gk, maxiter, False, nr

def NONLINEAR_CONJUGATE_GRADIENT_HS(xk: np.array, f, gradf, maxiter: int,
                                tol: float, a_init: float, p: float,
                                c1: float, c2: float, Nb: int):
    """
    THIS FINDS THE MINIMIZER OF f USING THE NON-LINEAR CONJUGATE GRADIENT METHOD WITH THE HESTENES-STIEFEL FORMULA.

    Args:
    - xk:      initial guess.
    - f:       function to minimize.
    - gradf:   gradient of the function to minimize.
    - maxiter: maximum number of iterations.
    - tol:     method tolerance.
    - a_init:  initial value for the step size in backtracking.
    - p:       reduction factor for the step size in backtracking.
    - c1:      parameter for the sufficient descent condition.
    - c2:      parameter for the curvature condition.
    - Nb:      maximum number of iterations in backtracking.
    
    Outputs:
    - xk:  approach to the minimizer of f.
    - gk:  gradient of f at xk.
    - k:   number of iterations.
    - T/F: if the method converged.
    - nr:  number of restarts (bk = 0).
    """
    gk = gradf(xk)
    dk = -gk
    nr = 0
    for k in range(maxiter + 1):
        if np.linalg.norm(gk) < tol:
            return xk, gk, k, True, nr
        ak, k1 = BACKTRAKING_WOLF(a_init = a_init, p = p, c1 = c1, c2 = c2,
                                xk = xk, f = f, gradf = gradf, dk = dk, Nb = Nb)
        xk = xk + ak*dk
        gk_n = gradf(xk)
        yk = gk_n - gk
        if np.abs(gk_n.T @ gk) < 0.2*np.linalg.norm(gk_n)**2:
            bk = (gk_n.T @ yk) / (dk.T @ yk)
        else:
            bk = 0
            nr += 1
        dk = -gk_n + bk*dk
        gk = gk_n
    return xk, gk, maxiter, False, nr

# **UNIDAD 6**

## **6.1**

In [ ]:
def GRADIENT(f, x: np.array, h: float):
    """
    COMPUTE THE GRADIENT OF A FUNCTION USING FORWARD FINITE DIFFERENCES.

    Args:
    - f: function to compute the gradient.
    - x: point to compute the gradient.
    - h: step size.

    Outputs:
    - grad: gradient of f at x.
    """
    n = len(x)
    grad = np.zeros(n)
    for i in range(n):
        ei = np.zeros(n)
        ei[i] = 1
        grad[i] = (f(x + h*ei) - f(x)) / h
    return grad

def CG(gk: np.array, Hk: np.array, n: int, tol: float):
    """
    CONJUGATE GRADIENT APPLIED TO NEWTON SYSTEM.

    Args:
    - gk:  gradient of f at xk.
    - Hk:  Hessian of f at xk.
    - n:   dimention.
    - tol: tolerance.

    Outputs:
    - pk: search direction.
    - j:  number of iterations.
    """
    zj = 0
    rj = gk
    dj = -rj
    for j in range(n-1):
        if dj.T @ Hk @ dj <= 0:
            if j == 0:
                pk = -gk
            else:
                pk = zj
        aj = (rj.T @ rj) / (dj.T @ Hk @ dj)
        zj = zj + aj * dj
        rj_n = rj + aj * Hk @ dj
        Bj = (rj_n.T @ rj_n) / (rj.T @ rj)
        dj = -rj_n + Bj * dj
        if np.linalg.norm(rj_n) < tol:
            pk = zj
            return pk, j
        rj = rj_n
    pk = zj
    return pk, n

## **6.2**

In [ ]:
def HESSIAN(f, x: np.array, h: float):
    """
    COMPUTE THE HESSIAN MATRIX OF A FUNCTION USING FINITE DIFFERENCES.

    Args:
    - f: function to compute the Hessian.
    - x: point to compute the Hessian.
    - h: step size.

    Outputs:
    - Hess: Hessian of f at x.
    """
    n = len(x)
    Hess = np.zeros((n,n))
    for i in range(n):
        ei = np.zeros(n)
        ei[i] = 1
        for j in range(n):
            ej = np.zeros(n)
            ej[j] = 1
            Hess[i,j] = (f(x + h*ei + h*ej) - f(x + h*ei) - f(x + h*ej) + f(x))/(h**2)
    return Hess

def NEWTON_CG_LINESEARCH_METHOD(f, gradf, Hessf, xk: np.array, tol: float, maxiter: int,
                                alpha_i: float, p: float, c: float, Nb: int):
    """
    NEWTON CONJUGATE GRADIENT METHOD WITH LINE SEARCH.

    Args:
    - f:       function to minimize.
    - gradf:   gradient of f.
    - Hessf:   Hessian of f.
    - xk:      initial point.
    - tol:     tolerance.
    - maxiter: maximum number of iterations.
    - alpha_i: initial alpha.
    - p:       reduction factor.
    - c:       constant for Armijo condition.
    - Nb:      maximum number of iterations for line search.

    Outputs:
    - xk:   optimal point.
    - gk:   gradient at xk.
    - k:    number of iterations.
    - T/F:  if the method converged.
    - MEAN: average number of iterations for line search.
    """
    n = len(xk)
    iteraciones = []
    for k in range(maxiter):
        gk = gradf(xk)
        if np.linalg.norm(gk) < tol:
            return xk, gk, k, True, np.mean(iteraciones)
        Hk = Hessf(xk)
        ek = min(0.5, np.sqrt(np.linalg.norm(gk)))*np.sqrt(np.linalg.norm(gk))
        pk, i = CG(gk, Hk, n, ek)
        ak = BACKTRAKING(alpha_i = alpha_i, p = p, c = c, xk = xk, f = f,
                            fxk=f(xk), gradfxk = gk, pk = pk, Nb = Nb)[0]
        iteraciones.append(i)
        xk = xk + ak * pk
    return xk, gk, maxiter, False, np.mean(iteraciones)

In [ ]:
def errorRelativo_grad(f, gradf, n: int, nt: int, h: float):
    """
    Print statistics of relative errors in the gradient calculation (analytical vs. numerical)
    using finite differences.
    
    Args:
    - f:     function to find the gradient.
    - gradf: analytical gradient of f.
    - n:     dimension.
    - nt:    number of tests.
    - h:     step size.
    """
    ve = np.zeros(nt)
    gf = lambda x: GRADIENT(f = f, x = x, h = h) # Funcion gradiente generada con autograd.
    for i in range(nt):
        x0  = np.random.randn(n)
        g0  = gradf(x0)
        ga  = gf(x0)
        ve[i] = np.linalg.norm(g0-ga) / np.linalg.norm(ga)
    print('ERRORES RELATIVOS EN EL CÁLCULO DEL GRADIENTE PARA h =', h)
    print('Min: %.2e   Media: %.2e    Max: %.2e' %(np.min(ve), np.mean(ve), np.max(ve)), '\n')

def errorRelativo_hess(f, Hessf, n: int , nt: int, h: float):
    """
    Print statistics of relative errors in the Hessian matrix calculation (analytical vs. numerical)
    using finite differences.
    
    Args:
    - f:     function to find the Hessian.
    - Hessf: analytical Hessian of f.
    - n:     dimension.
    - nt:    number of tests.
    - h:     step size.
    """
    ve = np.zeros(nt)
    Hf = lambda x: HESSIAN(f = f, x = x, h = h) # Funcion Hessiana generada con autograd.
    for i in range(nt):
        x0  = np.random.randn(n)
        H0  = Hessf(x0)
        Ha  = Hf(x0)
        ve[i] = np.linalg.norm(H0-Ha) / np.linalg.norm(Ha)
    print('ERRORES RELATIVOS EN EL CÁLCULO DE LA HESSIANA PARA h =', h)
    print('Min: %.2e   Media: %.2e    Max: %.2e' %(np.min(ve), np.mean(ve), np.max(ve)), '\n')

# **UNIDAD 8**

## **8.3**

In [ ]:
def BFGS_MOD(f, gradf, xk: np.array, tol: float, Hk: np.array,
             N: int, alpha_i, p: float, c: float, Nb: int):
    """
    BFGS METHOD WITH MODIFICATION FOR THE HESSIAN MATRIX.

    Args:
    - f:       function to minimize.
    - gradf:   gradient of the function.
    - xk:      initial point.
    - tol:     tolerance.
    - Hk:      initial Hessian matrix.
    - N:       maximum number of iterations.
    - alpha_i: initial step size.
    - p:       reduction factor for the step size.
    - c:       constant for the Armijo condition.
    - Nb:      maximum number of iterations for the backtracking line search.

    Returns:
    - xk:  optimal point.
    - gk:  gradient at the optimal point.
    - k:   number of iterations.
    - T/F: if the method converged.
    """
    n = len(xk)
    for k in range(N-1):
        gk = gradf(xk)
        if np.linalg.norm(gk) < tol:
            return xk, gk, k, True
        pk = - Hk @ gk
        if pk.T @ gk > 0:
            lb1 = 10**(-5) + (pk.T @ gk)/(gk.T @ gk)
            Hk = Hk + lb1*np.eye(n)
            pk = pk - lb1*gk
        ak = BACKTRAKING(alpha_i = alpha_i, p = p, c = c, xk = xk, f = f,
                        fxk = f(xk), gradfxk = gk, pk = pk, Nb = Nb)[0]
        xk_n = xk + ak * pk
        gk_n = gradf(xk_n)
        sk = xk_n - xk
        yk = gk_n - gk
        if yk.T @ sk <= 0:
            lb2 = 10**(-5) - (yk.T @ sk)/(yk.T @ yk)
            Hk = Hk + lb2*np.eye(n)
        else:
            rhok = 1/(yk.T @ sk)
            Hk = (np.eye(n) - rhok*np.outer(sk,yk)) @ Hk @ (np.eye(n) - rhok*np.outer(yk,sk)) + rhok*np.outer(sk,sk)
        xk = xk_n
    return xk, gk, N, False

## **8.4**

In [ ]:
def L_BFGS(f, gradf, xk: np.array, tol: float, Hk: np.array, m: int, max_iter: int):
    """
    L-BFGS METHOD.
    Args:
    - f:     function to minimize.
    - gradf: gradient of the function.
    - xk:    initial point.
    - tol:   tolerance.
    - Hk:    initial Hessian matrix.
    Outputs:
    -
    """
    gk = gradf(xk)
    for i in range(max_iter):
        pk = - Hk @ gk
        ak = 1
        xk_n = xk + ak * pk
        if i > m:
            break
        sk = xk_n - xk
        yk = gradf(xk_n) - gk
    #k = 0
    #while np.linalg.norm(gk) > tol:
    #    pk = - Hk @ gk
    #    ak = 1
    #    xk_n = xk + ak * pk
    #    if k > m:
    #        break
    #    sk = xk_n - xk
    #    yk = gradf(xk_n) - gk
    #    k += 1

# **UNIDAD 9**

## **9.1**

In [ ]:
def GAUSS_NEWTON_METHOD(f, R, J, xk: np.array, N: int,
                        tol: float, p: float, c: float, Nb: int) -> tuple:
    """
    Gauss-Newton method.

    Args:
    - f:   function to minimize.
    - R:   residual function.
    - J:   Jacobian matrix of R.
    - xk:  initial guess.
    - N:   maximum number of iterations.
    - tol: tolerance.
    - p:   reduction factor for the step size.
    - c:   constant for the Armijo condition.
    - Nb:  maximum number of iterations for the backtracking line search.

    Returns:
    - xk:  optimal point.
    - pk:  optimal step.
    - k:   number of iterations.
    - T/F: if the algorithm converges.

    """
    for k in range(N-1):
        Rk = R(xk)
        Jk = J(xk)
        fk = f(xk)
        gk = Jk.T @ Rk
        pk = np.linalg.solve(Jk.T @ Jk, -gk)
        if np.linalg.norm(pk) < tol:
            return xk, pk, k, True
        ak = BACKTRAKING(alpha_i = 1, p = p, c = c, xk = xk,
                         f = f, fxk = fk, gradfxk = gk, pk = pk,
                         Nb = Nb)[0]
        xk = xk + ak * pk
    return xk, pk, N, False

## **9.2**

# **UNIDAD 10**